In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = True
is_export = True

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

# Data Loading

In [ ]:
from mesh4d import obj3d
from mesh4d.analyse import crave

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)
mesh_ls = [crave.fix_pvmesh_disconnect(mesh) for mesh in mesh_ls]

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

---
# Registration Approach Selection


_P.S. Only activate one of the following code blocks in this section_

In [ ]:
"""
from mesh4d.regist import reg_rbf

obj4d_type = reg_rbf.Obj4d_RBF
export_folder = 'output/rbf/exclude/'
regist_kwargs = {}
"""

In [ ]:
# """
from mesh4d.regist import reg_ecpd

obj4d_type = reg_ecpd.Obj4d_ECPD
export_folder = 'output/ecpd/exclude/'
regist_kwargs = {
    'scale_rate': 100,
}
# """

---

# K-fold cross-verfication

In [ ]:
import time
import mesh4d
from mesh4d import kps
mesh4d.output_msg = False

briefs = []

for name in landmarks.markers.keys():
    # split dataset
    landmarks_test, landmarks_train = landmarks.split((name, ))

    # registration
    start_time = time.time()

    o4 = obj4d_type(
        fps=120 / stride,
        enable_rigid=False,
        enable_nonrigid=True,
    )
    o4.add_obj(*breast_ls)
    o4.load_markerset('landmarks_train', landmarks_train)
    o4.load_markerset('landmarks_test', landmarks_test)
    o4.regist('landmarks_train', **regist_kwargs)

    # computation time
    duration = time.time() - start_time
    print("computation time: {}".format(duration))

    # virtual key points tracking
    kps_source = landmarks_test.get_time_coord(0)
    o4.vkps_track(kps_source, start_id=0, name='vkps')
    
    # quantitative estimation
    vkps = o4.assemble_markerset(name='vkps')
    diff = kps.MarkerSet.diff(vkps, landmarks_test)

    if is_export:
        utils.save_pkl_object(diff, export_folder=export_folder, export_name='diff_{}'.format(name))

    brief_str = "{}: test error: {}".format(name, diff['diff_str'])
    print(brief_str)
    briefs.append(brief_str)

In [ ]:
import os

if is_export:
    brief_path = os.path.join(export_folder, 'k-fold.txt')

    with open(brief_path, 'w') as f:
        for text in briefs:
            f.write('{}\n'.format(text))  # python will convert \n to os.linesep

# Error distribution

In [ ]:
import os

files = os.listdir(export_folder)
files = [os.path.join(export_folder, f) for f in files if ('.pkl' in f) and ('marker' in f) and ('duration' not in f)]

diff_rbf_ls = []

for file in files:
    diff_rbf_ls.append(utils.load_pkl_object(file))

In [ ]:
import numpy as np

error_ls = []

for idx in (range(18)):
    error_ls.append(diff_rbf_ls[idx]['dist_mean'])

error_ls = np.array(error_ls)
print('mean error: {:.2f}'.format(np.mean(error_ls)))

In [ ]:
from mesh4d.analyse import visual

landmarks_init = landmarks_init = landmarks.get_frame_coord(1).get_points_coord()
visual.show_mesh_value_mask(mesh_clip_ls[1], landmarks_init, error_ls/10, k_nbr=100, show_edges=True)